In [ ]:
!pip install git+https://github.com/muellerzr/fastai2.git

In [ ]:
#default_exp inference.export

# inference.export
> This module contains the main functionality for extracting the transform parameters from DataLoaders

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.vision.all import *

## Vision

For an example we will look at the pets dataset. We will define a series of transforms in our Pipelines, and we will attempt to extract them.

In [ ]:
path = untar_data(URLs.PETS)
fnames = get_image_files(path/'images')
pat = r'(.+)_\d+.jpg$'
batch_tfms = [*aug_transforms(size=224, max_warp=0.8), Normalize.from_stats(*imagenet_stats)]
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
bs=64
dls = ImageDataLoaders.from_name_re(path, fnames, pat, batch_tfms=batch_tfms, 
                                   item_tfms=item_tfms, bs=bs)

In [ ]:
# Cell
def to_list(b):
    "Recursively make any `L()` or CategoryMap to list"
    def _inner(o): 
        if isinstance(o,L) or isinstance(o, CategoryMap):
            return list(o)
        elif isinstance(o, Tensor):
            return np.array(to_detach(o))
        else: return o
    for k in b.keys():
        b[k] = apply(_inner,b[k])
    return b

In [ ]:
#export
def _gen_dict(tfm):
    "Grabs the `attrdict` and transform name from `tfm`"
    tfm_dict = attrdict(tfm, *tfm.store_attrs.split(','))
    if 'partial' in tfm.name:
        tfm_name = tfm.name[1].split(' --')[0]
    else:
        tfm_name = tfm.name.split(' --')[0]
    return tfm_dict, tfm_name

In [ ]:
#export
def _make_tfm_dict(tfms, type_tfm=False):
    "Extracts transform params from `tfms`"
    tfm_dicts = {}
    for tfm in tfms:
        if hasattr(tfm, 'store_attrs') and not isinstance(tfm, AffineCoordTfm):
            if type_tfm or tfm.split_idx is not 0:
                tfm_dict,name = _gen_dict(tfm)
                tfm_dict = to_list(tfm_dict)
                tfm_dicts[name] = tfm_dict
    return tfm_dicts

In [ ]:
#hide
test_eq(len(_make_tfm_dict(dls.tfms, True)), 1)
ab_dict = _make_tfm_dict(dls.after_batch)
in_('Normalize', ab_dict.keys());
not in_('Flip', ab_dict.keys());
it_dict = _make_tfm_dict(dls.after_item)
in_('RandomResizedCrop', ab_dict.keys())
not in_('ToTensor', ab_dict.keys());

In [ ]:
#export
@typedispatch
def _extract_tfm_dicts(dl:TfmdDL):
    "Extracts all transform params from `dl`"
    type_tfm = True
    attrs = ['tfms','after_item','after_batch']
    tfm_dicts = {}
    for attr in attrs:
        tfm_dicts[attr] = _make_tfm_dict(getattr(dl, attr), type_tfm)
        if attr == 'after_item': tfm_dicts[attr]['ToTensor'] = True
        type_tfm = False
    return tfm_dicts

In [ ]:
#export
def get_information(dls): return _extract_tfm_dicts(dls[0])

### get_information

This function will take any set of `DataLoaders` and extract the transforms which are important during inference and their information

In [ ]:
tfm_info = get_information(dls)

In [ ]:
#hide
test_eq(len(tfm_info),3)
test_eq(tfm_info.keys(), ['tfms','after_item','after_batch'])

For vision it will contain `tfms`, `after_item`, and `after_batch`

First, our `type` transforms:

In [ ]:
tfm_info['tfms']

{'Categorize': {'add_na': False,
  'vocab': ['Abyssinian',
   'Bengal',
   'Birman',
   'Bombay',
   'British_Shorthair',
   'Egyptian_Mau',
   'Maine_Coon',
   'Persian',
   'Ragdoll',
   'Russian_Blue',
   'Siamese',
   'Sphynx',
   'american_bulldog',
   'american_pit_bull_terrier',
   'basset_hound',
   'beagle',
   'boxer',
   'chihuahua',
   'english_cocker_spaniel',
   'english_setter',
   'german_shorthaired',
   'great_pyrenees',
   'havanese',
   'japanese_chin',
   'keeshond',
   'leonberger',
   'miniature_pinscher',
   'newfoundland',
   'pomeranian',
   'pug',
   'saint_bernard',
   'samoyed',
   'scottish_terrier',
   'shiba_inu',
   'staffordshire_bull_terrier',
   'wheaten_terrier',
   'yorkshire_terrier']}}

Then the `item` transforms:

In [ ]:
tfm_info['after_item']

{'RandomResizedCrop': {'min_scale': 0.75,
  'ratio': (1.0, 1.0),
  'size': (460, 460),
  'val_xtra': 0.14},
 'ToTensor': True}

And finally our batch transforms:

In [ ]:
tfm_info['after_batch']

{'IntToFloatTensor': {'div': 255.0, 'div_mask': 1},
 'Normalize': {'axes': (0, 2, 3), 'mean': array([[[[0.485]],
  
          [[0.456]],
  
          [[0.406]]]], dtype=float32), 'std': array([[[[0.229]],
  
          [[0.224]],
  
          [[0.225]]]], dtype=float32)}}

## Tabular

Next we'll look at a tabular example. We will use the `ADULT_SAMPLE` dataset here:

In [ ]:
#export
from fastai2.tabular.all import *

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)

In [ ]:
df = pd.read_csv(path/'adult.csv')
splits = RandomSplitter()(range_of(df))
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
y_names = 'salary'

In [ ]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, splits=splits)
dls = to.dataloaders()

In [ ]:
#export
@typedispatch
def _extract_tfm_dicts(dl:TabDataLoader):
    "Extracts all transform params from `dl`"
    types = 'normalize,fill_missing,categorify'
    if hasattr(dl, 'categorize'): types += ',categorize'
    if hasattr(dl, 'regression_setup'): types += ',regression_setup'
    tfms = {}
    for t in types.split(','):
        tfm = getattr(dl, t)
        tfms[t] = to_list(attrdict(tfm, *tfm.store_attrs.split(',')))
    return tfms

The usage is the exact same:

In [ ]:
tfm_dicts = get_information(dls)

In [ ]:
#hide
test_eq(len(tfm_dicts),4)

However our keys are different. By default it will have `normalize`, `fill_missing`, and `categorify`, and then depending on what is available it will store either `categorize` or `regression_setup` to tell us about our outputs.

Here is an example from `normalize`:

In [ ]:
tfm_dicts['normalize']

{'means': {'age': 38.61818112019655,
  'education-num': 10.08330454144113,
  'fnlwgt': 189097.24162155937},
 'stds': {'age': 13.65958366512057,
  'education-num': 2.5547649904806056,
  'fnlwgt': 105240.85724028994}}

`fill_missing`:

In [ ]:
tfm_dicts['fill_missing']

{'add_col': True,
 'fill_strategy': 'median',
 'fill_vals': {'education-num': 0},
 'na_dict': {'education-num': 10.0}}

And `categorify`:

In [ ]:
tfm_dicts['categorify']

{'classes': {'education': ['#na#',
   ' 10th',
   ' 11th',
   ' 12th',
   ' 1st-4th',
   ' 5th-6th',
   ' 7th-8th',
   ' 9th',
   ' Assoc-acdm',
   ' Assoc-voc',
   ' Bachelors',
   ' Doctorate',
   ' HS-grad',
   ' Masters',
   ' Preschool',
   ' Prof-school',
   ' Some-college'],
  'education-num_na': ['#na#', False, True],
  'marital-status': ['#na#',
   ' Divorced',
   ' Married-AF-spouse',
   ' Married-civ-spouse',
   ' Married-spouse-absent',
   ' Never-married',
   ' Separated',
   ' Widowed'],
  'occupation': ['#na#',
   ' ?',
   ' Adm-clerical',
   ' Armed-Forces',
   ' Craft-repair',
   ' Exec-managerial',
   ' Farming-fishing',
   ' Handlers-cleaners',
   ' Machine-op-inspct',
   ' Other-service',
   ' Priv-house-serv',
   ' Prof-specialty',
   ' Protective-serv',
   ' Sales',
   ' Tech-support',
   ' Transport-moving'],
  'race': ['#na#',
   ' Amer-Indian-Eskimo',
   ' Asian-Pac-Islander',
   ' Black',
   ' Other',
   ' White'],
  'relationship': ['#na#',
   ' Husband',
   

Before finally `categorize` (since we have a classification problem):

In [ ]:
tfm_dicts['categorize']

{'add_na': False, 'vocab': ['<50k', '>=50k']}

## Exporting

To export, a new `to_fastinference` function has been made

In [ ]:
#export
@patch
def to_fastinference(x:Learner, data_fname='data', model_fname='model', path=Path('.')):
    "Export data for `fastinference_onnx` or `_pytorch` to use"
    dicts = get_information(x.dls)
    with open(path/f'{data_fname}.pkl', 'wb') as handle:
        pickle.dump(procs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    x._end_cleanup()
    state = x.opt.state_dict() if x.opt is not None else None
    x.opt = None
    torch.save(x.model.state_dict(), path/f'{model_fname}.pkl')

In [ ]:
doc(Learner.to_fastinference)

Learner.to_fastinference(x:fastai2.learner.Learner, data_fname='data', model_fname='model', path=Path('.'))
Export data for `fastinference_onnx` or `_pytorch` to use

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev


Params:

* `data_fname`: Filename to save our extracted `DataLoader` information, default is `data`
* `model_fname`: Filename to save our current model, default is `model`
* `path`: Path to save our model and data to, default is `.`

Exported files will have the extension `.pkl`

In [ ]:
learn = tabular_learner(dls, layers=[200,100])

Simply call `learn.to_fastinference` and it will export everything needed for `fastinference_pytorch` or `fastinference_onnx`

In [ ]:
learn.to_fastinference(data_fname = 'data', model_fname = 'model', path = Path('.'))

In [ ]:
#hide
"""
# TODO: Text
Things to save:


* `data.vocab`
* `data.o2i`
* Tokenizer
* All the rules in `text.core`:
[<function fastai2.text.core.fix_html>,
 <function fastai2.text.core.replace_rep>,
 <function fastai2.text.core.replace_wrep>,
 <function fastai2.text.core.spec_add_spaces>,
 <function fastai2.text.core.rm_useless_spaces>,
 <function fastai2.text.core.replace_all_caps>,
 <function fastai2.text.core.replace_maj>,
 <function fastai2.text.core.lowercase>]

- Ensure that `L` is in the library
"""